Now that you understand what a combinatorial test is, let’s see in practice how it is done. In our tutorial we'll be using a tool for generating combinatorial parameters called [Covertable](https://github.com/walkframe/covertable/blob/master/python/README.rst) and a testing library for Python called [Pytest](https://github.com/pytest-dev/pytest).

## Motivation
Let’s start by looking at the file code.py which has the code we want to test

```python
def important_function(pressure, volume, velocity, low_fuel):
    if pressure < 10:
        if volume > 300:
            if velocity == 5:
                do_something_bad()
        elif low_fuel:
            do_something_good()
    else:
        do_something_good()

def do_something_good():
    pass

def do_something_bad():
    raise Exception("A bug happened!")
```



We want to test important_function, which takes three integer parameters and a boolean. Looking at the code, we can see that there’s a bug (do_something_bad) in one of the function’s branches. But the bug only happens if the three parameters satisfy some conditions: `pressure < 10`, `volume > 300`, and `velocity == 5`. So the bug is a result of a three-way interaction, because it happens only for a specific combination of those three parameters.

## Executing test set with Pytest and Covertable
Now that we took a look in the code that we want to test, let's see how can we actually use Covertable and Pytest to generate the combination of parameters that will spot the bug.

First, we need to install both pytest and covertable dependencies:

In [4]:
%%bash

pip install pytest
pip install covertable

Now that we have the dependencies installed, let's take a look in the file `test_parameterized.py`, which implements a test using those libraries.

```python
import pytest
from covertable import make

@pytest.mark.parametrize(["pressure", "volume", "velocity", "low_fuel"],
    make([[5,10,15],
        [200, 300, 400],
        [1, 2, 3, 4, 5],
        [True, False]], length=3)
)
def test_important_function(pressure, volume, velocity, low_fuel):
    important_function(pressure, volume, velocity, low_fuel)



As you can see, `pytest` provides us the annotation `@pytest.mak.parametrize` that receives two arguments: an array of parameters names and a array of array of values for each of those parameters. The function below this annotation will be tested for each of the values specified in the second argument.



In the first argument, we need to provide the name of the parameters we want to test. In this case, as we want to test `important_function`, we need to pass `["pressure", "volume", "velocity", "low_fuel"]`.

In the second argument, we need to pass an array of array of values, that would be the combination of parameters given to `important_function`. To do that, we will be using `covertable` function called `make`. This function receives an array of array of values and return an array with the combination of them. To be more clear, as we passed `[5,10,15]` as the first element of the array, those are the possible values for the parameter `pressure`, `[200, 300, 400]` would be the possible values for the parameter `volume` and so on.

It's important to notice that `make` also accepts some configuration. In the second argument, we set `length=3`. This is the degree of interaction that we teached earlier in this article. As we need the combination of `pressure`, `volume`, and `velocity` to catch the bug, our degree of interaction will be 3
(in a real-world scenario, though, we wouldn’t know when a bug happens. So, you would want to choose a degree of interaction that gives you an appropriate level of confidence, as we discussed).

Now that we have our test file, we just need to run pytest and see if there's a bug:

In [5]:
%%bash

python -m pytest

============================= test session starts ==============================
platform linux -- Python 3.6.9, pytest-3.6.4, py-1.9.0, pluggy-0.7.1
rootdir: /content, inifile:
plugins: typeguard-2.7.1
collected 45 items

sample_data/test_parameterized.py ...........F.......................... [ 84%]
.......                                                                  [100%]

=================================== FAILURES ===================================
____________________ test_important_function[5-400-5-False] ____________________

pressure = 5, volume = 400, velocity = 5, low_fuel = False

    @pytest.mark.parametrize(["pressure", "volume", "velocity", "low_fuel"],
        make([[5,10,15],
            [200, 300, 400],
            [1, 2, 3, 4, 5],
            [True, False]], length=3)
    )
    def test_important_function(pressure, volume, velocity, low_fuel):
>       important_function(pressure, volume, velocity, low_fuel)

sample_data/test_parameterized.py:27: 
_ _ _ _ _ _ _

When running this command, you will see the log of parameters used to test the function and spot the bug caused by the combination of values that satisfies `pressure < 10, volume > 300, and velocity == 5`.